In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import skimage
import tqdm
import skimage.morphology
import ipywidgets as widgets
import json

from ipywidgets import interact, interact_manual
from scipy.optimize import basinhopping
from scipy import optimize

from pathlib import Path

from IPython.display import clear_output

In [ ]:
data_path = Path('/data/Datasets/usg-kaggle/train/')

In [ ]:
def get_bouncing_coordinate(coordinate_value, limit):
    if coordinate_value >= 0: 
        if (coordinate_value // limit) % 2 == 0:
            return coordinate_value % limit
        return limit - (coordinate_value % limit) - 1
    else:
        if (abs(coordinate_value) // limit) % 2 == 0:
            return limit - (coordinate_value % limit) -1
        return coordinate_value % limit

In [ ]:
def get_pixel_values(center, angle, image):
    x_c, y_c = center
    a = np.tan(np.deg2rad(angle))
    b = y_c - (a * x_c)
    mask = np.zeros_like(image)
    x1 = min(max(-b / (a + 1e-8), 0), image.shape[1])
    y1 = int(a * x1 + b)
    x1 = int(x1)
    
    start_point = np.asarray([[y1, x1]])
    
    x2 = min(max((image.shape[1] - b) / (a + 1e-8), 0), image.shape[1])
    y2 = int(a * x2 + b)
    x2 = int(x2)
    
    mask = np.zeros_like(image)
    cv2.line(mask, (x1, y1), (x2, y2), 1, 1, cv2.LINE_AA)
    points = np.stack(np.where(mask == 1), axis=-1)
    values = []
    kernel_size = 3
    for y, x in points:
        a_sum = 0
        for of_y in range(-kernel_size // 2, kernel_size // 2 + 1):
            for of_x in range(-kernel_size // 2, kernel_size // 2 + 1):
                a_sum += image[
                    get_bouncing_coordinate(y + of_y, image.shape[0]), 
                    get_bouncing_coordinate(x + of_x, image.shape[1])
                ]
        values.append(a_sum)
    
    distances = np.sqrt(np.sum((points - start_point) ** 2, axis=1))
    return values, distances, mask

In [ ]:
directories = [path for path in data_path.glob('*') if path.is_dir()]
dir_widget = widgets.Dropdown(
    options=directories,
    index=0,
    description="Directory:"
)

files = list([path for path in Path(dir_widget.value).rglob('lower.png') if not path.name.startswith('.')])
files_widget = widgets.Dropdown(
    options=files,
    index=0,
    description="File:"
)

angle_widget = widgets.IntSlider(
    min=0,
    max=359,
    value=90,
    description="Angle:"
)


@interact(a_dir=dir_widget, a_file=files_widget, angle=angle_widget)
def get_img(a_dir, a_file, angle):
    with files_widget.hold_trait_notifications():
        files = list([path for path in Path(dir_widget.value).rglob('lower.png') if not path.name.startswith('.')])
        files = set(files)
        if len(files.difference(set(files_widget.options))) > 0:
            files_widget.options = files
        
    img_file = cv2.imread(a_file.as_posix(), cv2.IMREAD_GRAYSCALE)
    img_file = ((img_file.astype('float32') / 255) ** 8)
    img_file = ((img_file - img_file.min()) / (img_file.max() - img_file.min()) * 255).astype('uint8')
    coords = json.loads((a_file.parent / "coordinates.json").read_text())
    circle_coords = coords["circle"]
    
    mask = np.zeros_like(img_file)
    radia = []
    sums = []
    circle_center = circle_coords["x"], circle_coords["y"]
    for i in range(5, 50):
        cv2.circle(mask, circle_center, i, 255, 1)
        sums.append(img_file[mask == 255].sum() / (2 * np.pi * i))
        radia.append(i)
        mask[:] = 0
        
    gradients = np.diff(sums)
    radia = radia[:-1]
    ascendence = radia[np.argmax(gradients)]
    descendence = radia[np.argmin(gradients)]
    
    orig = img_file.copy()
    img_file = cv2.medianBlur(img_file, 3)
    values, points, mask = get_pixel_values(circle_center, angle, img_file)
    fig, ax = plt.subplots(1, 3, figsize=(18, 4))
    ax[0].plot(radia, gradients)
    ax[1].plot(points, values)
    ax[2].plot(points[1:], np.diff(values))
    
    fig, ax = plt.subplots(1, 2, figsize=(18.6 * 2, 18.6), sharey=True)
    ax[0].imshow(orig, cmap='gray')
    ax[1].imshow(mask, cmap="gray")
    fig.tight_layout()
    
    le_image = np.abs(orig.astype(np.float32) - img_file.astype(np.float32))
    le_image = (le_image - le_image.min()) / (le_image.max() - le_image.min())
    le_image[le_image > np.percentile(le_image, 98)] = 1
    le_image[le_image <= np.percentile(le_image, 98)] = 0
    
    kernel = np.ones((1,1),np.uint8)
    le_image = cv2.medianBlur(le_image, 3)
    
    plt.figure(figsize=(18.6, 18.6))
    plt.imshow(le_image)